In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
review1 = pd.read_csv("data.csv")
review1.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [4]:
review1.drop_duplicates(inplace=True)

In [5]:
review1.dropna(inplace=True)

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+','',text)
    text = re.sub(r'[^\w\s]','',text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(words).strip()
    return text

In [7]:
review1['cleaned_text']=review1['Review text'].apply(clean_text)

In [8]:
cleaned_df = review1

In [9]:
def assign_ratings(ratings):
    if ratings == 5 or ratings == 4:
        return "Positive"
    else:
        return "Negative"


cleaned_df["Target"]=cleaned_df["Ratings"].apply(assign_ratings)

In [10]:
cleaned_df['Target'] = cleaned_df['Target'].replace({'Positive':1,'Negative':0}, regex=True)

C:\Users\Manisha\AppData\Local\Temp\ipykernel_12508\2910730347.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_df['Target'] = cleaned_df['Target'].replace({'Positive':1,'Negative':0}, regex=True)


In [11]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import Pipeline

from sklearn import metrics
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [12]:
tfidf = TfidfVectorizer()

In [13]:
x=tfidf.fit_transform(cleaned_df['cleaned_text']).toarray()
y=cleaned_df['Target']

In [14]:
x=tfidf.fit_transform(cleaned_df['cleaned_text']).toarray()
y=cleaned_df['Target']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [16]:
rf=RandomForestClassifier()

In [16]:
import mlflow

In [18]:
mlflow.set_experiment('experiment_names')

<Experiment: artifact_location='file:///C:/Users/Manisha/machine%20learning/mlops_pipelines/mlruns/577749130137172473', creation_time=1756962275017, experiment_id='577749130137172473', last_update_time=1756962275017, lifecycle_stage='active', name='experiment_names', tags={}>

In [19]:
mlflow.sklearn.autolog()

In [20]:
with mlflow.start_run() as run:
    rf.fit(x_train, y_train)

2025/09/04 12:48:27 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2025/09/04 12:48:27 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Trackin

In [17]:
mlflow.set_experiment('sentiment_prediction')

<Experiment: artifact_location='file:///C:/Users/Manisha/machine%20learning/mlops_pipelines/mlruns/602232716195977226', creation_time=1756962531733, experiment_id='602232716195977226', last_update_time=1756962531733, lifecycle_stage='active', name='sentiment_prediction', tags={}>

In [22]:
pipe1 = Pipeline(
    [
        ('classifier', RandomForestClassifier())
    ]
)

In [23]:
param_grid1 = {
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__max_depth': [None, 5, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt', 'log2']
}

In [24]:
# try only 20 random combinations
clf = RandomizedSearchCV(
    estimator=pipe1,
    param_distributions=param_grid1,
    n_iter=20,             
    cv=5,
    verbose=2,
    n_jobs=-1
)

In [25]:
mlflow.sklearn.autolog(max_tuning_runs=None)

In [26]:
with mlflow.start_run() as run:
    %time clf.fit(x_train, y_train)

2025/09/04 12:51:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 20 candidates, totalling 100 fits


2025/09/04 12:56:10 WARNING mlflow.sklearn: Encountered exception during creation of child runs for parameter search. Child runs may be missing. Exception: 'NoneType' object has no attribute '_to_mlflow_entity'


CPU times: total: 20 s
Wall time: 4min 16s


In [18]:
pipelines = {
    'knn': Pipeline([
        ('classifier', KNeighborsClassifier())
    ]), 
    'svc': Pipeline([
        ('classifier', SVC())
    ]),
    'logistic_regression': Pipeline([
        ('classifier', LogisticRegression(max_iter=5000))
    ]),
    'random_forest': Pipeline([
        ('classifier', RandomForestClassifier())
    ]),
    'decision_tree': Pipeline([
        ('classifier', DecisionTreeClassifier())
    ]),
    'naive_bayes': Pipeline([
        ('classifier', GaussianNB())
    ])
}

In [19]:
# Parameter grids
parameter_grids = {
    'knn': [{
        'classifier__n_neighbors': [i for i in range(1, 20, 2)], 
        'classifier__p': [1, 2, 3]
    }],
    'svc': [
        {
            'classifier__kernel': ['rbf'], 
            'classifier__C': [0.01, 0.1, 1, 10, 100]
        }, 
        {
            'classifier__kernel': ['poly'], 
            'classifier__degree': [2, 3, 4, 5], 
            'classifier__C': [0.01, 0.1, 1, 10, 100]
        }, 
        {
            'classifier__kernel': ['linear'], 
            'classifier__C': [0.01, 0.1, 1, 10, 100]
        }
    ],
    'logistic_regression': [
        {
            'classifier__C': [0.1, 1, 10], 
            'classifier__penalty': ['l2']
        }, 
        {
            'classifier__C': [0.1, 1, 10], 
            'classifier__penalty': ['l1'], 
            'classifier__solver': ['liblinear']
        }, 
        {
            'classifier__C': [0.1, 1, 10], 
            'classifier__penalty': ['elasticnet'], 
            'classifier__l1_ratio': [0.4, 0.5, 0.6],
            'classifier__solver': ['saga']
        }
    ],
    'random_forest': [{
        'classifier__n_estimators': [50, 100, 200]
    }],
    'decision_tree': [{
        'classifier__max_depth': [None, 5, 10]
    }],
    'naive_bayes': [{}]   # no hyperparameters to tune
}

In [20]:
# Perform ramdomizedSearchCV for each algorithm
best_models_sentiment_prediction = {}

In [30]:
for algorithm in pipelines.keys():
    print("-" * 10, algorithm, "-" * 10)
    random_search = RandomizedSearchCV(
        estimator=pipelines[algorithm],
        param_distributions=parameter_grids[algorithm],
        n_iter=10,
        cv=5,
        verbose=2,
        n_jobs=-1
    )

    mlflow.sklearn.autolog(max_tuning_runs=None)

    with mlflow.start_run() as run:
        %time random_search.fit(x_train, y_train)

    print('Train Score: ', random_search.best_score_)
    print('Test Score: ', random_search.score(x_test, y_test))

    best_models_sentiment_prediction[algorithm] = random_search.best_estimator_
    print()

---------- knn ----------


2025/09/04 12:56:35 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 10 candidates, totalling 50 fits


2025/09/04 13:34:42 WARNING mlflow.sklearn: Encountered exception during creation of child runs for parameter search. Child runs may be missing. Exception: 'NoneType' object has no attribute '_to_mlflow_entity'


CPU times: total: 1min 51s
Wall time: 38min 6s
Train Score:  0.8641185647425896
Test Score:  0.8658764815970056

---------- svc ----------


...

CPU times: total: 6.95 s
Wall time: 1min 14s
Train Score:  0.8567862714508581
Test Score:  0.8665003119151591

---------- naive_bayes ----------


2025/09/04 14:19:52 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 1 candidates, totalling 5 fits


2025/09/04 14:20:18 WARNING mlflow.sklearn: Encountered exception during creation of child runs for parameter search. Child runs may be missing. Exception: 'NoneType' object has no attribute '_to_mlflow_entity'


CPU times: total: 7.14 s
Wall time: 26.5 s
Train Score:  0.33057722308892357
Test Score:  0.3237679351216469



In [21]:
# Stop the auto logger

mlflow.sklearn.autolog(disable=True)

In [22]:
import time
import joblib
import os

In [23]:
dev = "Rushi Chavan"
best_models = {}

In [24]:
for algorithm in pipelines.keys():
    print("-" * 10, algorithm, "-" * 10)
    random_search = RandomizedSearchCV(
        estimator=pipelines[algorithm],
        param_distributions=parameter_grids[algorithm],
        n_iter=10,
        cv=5,
        verbose=2,
        n_jobs=-1
    )

    # Fit
    start_fit_time = time.time()
    random_search.fit(x_train, y_train)
    end_fit_time = time.time()

    # Predict
    start_predict_time = time.time()
    y_pred = random_search.predict(x_test)
    end_predict_time = time.time()

    # Saving the best model
    joblib.dump(random_search.best_estimator_, f'best_models/{algorithm}.pkl')
    model_size = os.path.getsize(f'best_models/{algorithm}.pkl')

    # Pring Log
    print('Train Score: ', random_search.best_score_)
    print('Test Score: ', random_search.score(x_test, y_test))
    print("Fit Time: ", end_fit_time - start_fit_time)
    print("Predict Time: ", end_predict_time - start_predict_time)
    print("Model Size: ", model_size)

    print()

    # Start the experiment run
    with mlflow.start_run() as run:
        # Log tags with mlflow.set_tag()
        mlflow.set_tag("developer", dev)

        # Log Parameters with mlflow.log_param()
        mlflow.log_param("algorithm", algorithm)
        mlflow.log_param("hyperparameter_grid", parameter_grids[algorithm])
        mlflow.log_param("best_hyperparameter", random_search.best_params_)

        # Log Metrics with mlflow.log_metric()
        mlflow.log_metric("train_score", random_search.best_score_)
        mlflow.log_metric("test_score", random_search.score(x_test, y_test))
        mlflow.log_metric("fit_time", end_fit_time - start_fit_time)
        mlflow.log_metric("predict_time", end_predict_time - start_predict_time)
        mlflow.log_metric("model_size", model_size)

        # Log Model using mlflow.sklearn.log_model()
        mlflow.sklearn.log_model(random_search.best_estimator_, f"{algorithm}_model")

---------- knn ----------
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Train Score:  0.8647425897035881
Test Score:  0.8658764815970056
Fit Time:  318.3239917755127
Predict Time:  3.4168448448181152
Model Size:  177532258



2025/09/05 13:34:08 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2025/09/05 13:34:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instea

---------- svc ----------
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Train Score:  0.8720748829953198
Test Score:  0.8820960698689956
Fit Time:  1909.033476114273
Predict Time:  20.120227813720703
Model Size:  81518689



2025/09/05 14:08:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 14:08:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


---------- logistic_regression ----------
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Train Score:  0.8716068642745709
Test Score:  0.875857766687461
Fit Time:  266.8124511241913
Predict Time:  0.06249594688415527
Model Size:  28661



...

2025/09/05 14:17:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 14:17:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


---------- decision_tree ----------
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Train Score:  0.8570982839313575
Test Score:  0.8671241422333126
Fit Time:  54.65463709831238
Predict Time:  0.015618324279785156
Model Size:  10887



2025/09/05 14:18:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 14:18:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


---------- naive_bayes ----------
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Train Score:  0.33057722308892357
Test Score:  0.3237679351216469
Fit Time:  3.918607473373413
Predict Time:  0.17183685302734375
Model Size:  111621



2025/09/05 14:18:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 14:18:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
